# Badanie danych rzeczywistych
Dane przedstawione w materiale edukacyjnym są często niezwykle doskonałe, zaprojektowane tak, aby pokazać uczniom, jak znaleźć jasne relacje między zmiennymi. Dane "świata rzeczywistego" są nieco mniej proste.

Ze względu na złożoność danych "rzeczywistych" musimy sprawdzić nieprzetworzone dane pod kątem problemów przed ich użyciem.

W związku z tym najlepszym rozwiązaniem jest sprawdzenie pierwotnych danych i przetworzenie ich przed użyciem, co zmniejsza błędy lub problemy, zwykle usuwając błędne punkty danych lub modyfikując dane w bardziej użyteczny formularz.

## Rzeczywiste problemy z danymi
Rzeczywiste dane mogą zawierać wiele różnych problemów, które mogą mieć wpływ na użyteczność danych i naszą interpretację wyników.

Ważne jest, aby pamiętać, że na większość rzeczywistych danych wpływa czynniki, które nie zostały zarejestrowane w tym czasie. Na przykład możemy mieć tabelę czasów toru wyścigowego obok rozmiarów silnika; ale różne inne czynniki, które nie zostały zapisane, takie jak pogoda, prawdopodobnie również odegrały rolę. W przypadku problemów często możemy zmniejszyć wpływ tych czynników, zwiększając rozmiar zestawu danych.

W innych sytuacjach punkty danych, które są wyraźnie poza oczekiwanymi wartościami — znanymi również jako "wartości odstające" — czasami mogą być bezpiecznie usuwane z analiz, chociaż musimy zadbać o to, aby nie usuwać punktów danych, które zapewniają rzeczywiste szczegółowe informacje.

Innym typowym problemem w rzeczywistych danych jest stronnicza odchylenie. Stronniczość odnosi się do tendencji do wybierania niektórych typów wartości częściej niż inne w sposób, który błędnie przedstawia podstawową populację lub "rzeczywisty świat". Stronniczy może być czasami identyfikowany przez eksplorowanie danych, mając na uwadze podstawową wiedzę na temat tego, skąd pochodzą dane.

Rzeczywiste dane zawsze będą miały problemy, ale analitycy danych często mogą przezwyciężyć następujące problemy:

- Sprawdzanie brakujących wartości i źle zarejestrowanych danych.
- Rozważając usunięcie oczywistych wartości odstających.
- Badanie, jakie rzeczywiste czynniki mogą mieć wpływ na ich analizę i określenie, czy ich rozmiar zestawu danych jest wystarczająco duży, aby zmniejszyć wpływ tych - czynników.
- Sprawdzanie stronniczych danych pierwotnych i rozważenie ich opcji naprawy stronniczych, jeśli zostanie znaleziona.

# Eksploracja danych w Pythonie — dane rzeczywiste

W tym notatniku zbadasz rozkłady danych, które nie są normalne, oraz nauczysz się wykonywać podstawowe porównania między zmiennymi.

## Ładowanie danych

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

# Load data from a text file
df_students = pd.read_csv('grades.csv',delimiter=',',header='infer')

# Remove any rows with missing data
df_students = df_students.dropna(axis=0, how='any')

# Calculate who passed, assuming '60' is the grade needed to pass
passes  = pd.Series(df_students['Grade'] >= 60)

# Save who passed to the Pandas dataframe
df_students = pd.concat([df_students, passes.rename("Pass")], axis=1)


# Print the result out into this notebook
print(df_students)


# Create a function that we can re-use
def show_distribution(var_data):
    '''
    This function will make a distribution (graph) and display it
    '''

    # Get statistics
    min_val = var_data.min()
    max_val = var_data.max()
    mean_val = var_data.mean()
    med_val = var_data.median()
    mod_val = var_data.mode()[0]

    print('Minimum:{:.2f}\nMean:{:.2f}\nMedian:{:.2f}\nMode:{:.2f}\nMaximum:{:.2f}\n'.format(min_val,
                                                                                            mean_val,
                                                                                            med_val,
                                                                                            mod_val,
                                                                                            max_val))

    # Create a figure for 2 subplots (2 rows, 1 column)
    fig, ax = plt.subplots(2, 1, figsize = (10,4))

    # Plot the histogram   
    ax[0].hist(var_data)
    ax[0].set_ylabel('Frequency')

    # Add lines for the mean, median, and mode
    ax[0].axvline(x=min_val, color = 'gray', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=mean_val, color = 'cyan', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=med_val, color = 'red', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=mod_val, color = 'yellow', linestyle='dashed', linewidth = 2)
    ax[0].axvline(x=max_val, color = 'gray', linestyle='dashed', linewidth = 2)

    # Plot the boxplot   
    ax[1].boxplot(var_data, vert=False)
    ax[1].set_xlabel('Value')

    # Add a title to the Figure
    fig.suptitle('Data Distribution')

    # Show the figure
    fig.show()


show_distribution(df_students['Grade'])

Jak być może pamiętasz, nasze dane miały średnią i dominantę w centrum, a wartości rozkładały się symetrycznie wokół środka.

Teraz przyjrzyjmy się rozkładowi danych dotyczących czasu nauki.

In [ ]:
# Get the variable to examine
col = df_students['StudyHours']
# Call the function
show_distribution(col)

Rozkład czasu nauki różni się znacząco od rozkładu ocen.

Zauważ, że wąsy na wykresie box zaczynają się dopiero około 6.0, co oznacza, że przeważająca część pierwszego kwartylu danych jest powyżej tej wartości. Minimum oznaczone jest symbolem **o**, co sugeruje, że może być to statystyczny *odstający* (ang. *outlier*): wartość znacząco odbiegająca od reszty rozkładu.

Dane odstające mogą wystąpić z różnych powodów. Może student chciał wpisać "10" godzin, ale wpisał "1" i pominął "0". Albo może naprawdę poświęcił bardzo mało czasu na naukę. W każdym razie to anomalia statystyczna, która nie odzwierciedla typowego przypadku. Spójrzmy, jak wygląda rozkład bez tej wartości.

In [ ]:
# Get the variable to examine
# We will only get students who have studied more than one hour
col = df_students[df_students.StudyHours>1]['StudyHours']

# Call the function
show_distribution(col)

Dla celów dydaktycznych potraktowaliśmy wartość **1** jako prawdziwy odstający i ją wyłączyliśmy. W praktyce jednak rzadko usuwa się wartości skrajne bez dodatkowego uzasadnienia, szczególnie gdy rozmiar próbki jest niewielki. Im mniejsza próbka, tym większe ryzyko, że nie odzwierciedla ona populacji (tu: wszystkich studentów, nie tylko naszych 22). Jeśli pobralibyśmy próbkę 1000 studentów, mogłoby się okazać, że krótkie czasy nauki są całkiem powszechne.

Gdy mamy więcej danych, próbka staje się bardziej wiarygodna i łatwiej jest traktować odstające wartości jako obserwacje poza określonymi percentylami, w których znajduje się większość danych. Na przykład poniższy kod używa funkcji **quantile** z Pandas, aby wykluczyć obserwacje poniżej 0.01-tego percentyla (wartość, powyżej której znajduje się 99% danych).

In [ ]:
# calculate the 0.01th percentile
q01 = df_students.StudyHours.quantile(0.01)
# Get the variable to examine
col = df_students[df_students.StudyHours>q01]['StudyHours']
# Call the function
show_distribution(col)

> **Wskazówka**: Możesz też usunąć wartości odstające na górnym końcu rozkładu, definiując próg jako wysoki percentyl. Na przykład funkcja **quantile** pozwala znaleźć 0.99 percentyl, poniżej którego znajduje się 99% danych.

Po usunięciu wartości odstających wykres box pokazuje dane mieszczące się w czterech kwartylach. Zauważ, że rozkład nie jest symetryczny jak w przypadku ocen. Pojawiają się wartości bardzo wysokie — około 16 godzin — ale większość danych mieści się między 7 a 13 godzinami. Kilka ekstremalnie wysokich wartości przesuwa średnią w stronę wyższych wartości.

Spójrzmy teraz na gęstość tego rozkładu.

In [ ]:
def show_density(var_data):
    fig = plt.figure(figsize=(10,4))

    # Plot density
    var_data.plot.density()

    # Add titles and labels
    plt.title('Data Density')

    # Show the mean, median, and mode
    plt.axvline(x=var_data.mean(), color = 'cyan', linestyle='dashed', linewidth = 2)
    plt.axvline(x=var_data.median(), color = 'red', linestyle='dashed', linewidth = 2)
    plt.axvline(x=var_data.mode()[0], color = 'yellow', linestyle='dashed', linewidth = 2)

    # Show the figure
    plt.show()

# Get the density of StudyHours
show_density(col)

Taki rozkład nazywa się *prawoskośnym* (right-skewed). Większość danych skupia się po lewej stronie rozkładu, a długi ogon po prawej wynika z ekstremalnie wysokich wartości, które przesuwają średnią w prawo.

**Miary zmienności**

Mamy już ogólne pojęcie, gdzie znajduje się środek rozkładów ocen i czasu nauki. Teraz warto sprawdzić inny aspekt — jak bardzo dane są rozproszone?

Typowe statystyki mierzące zmienność obejmują:

- **Zakres (Range)**: Różnica między maksimum a minimum. Nie ma wbudowanej funkcji o tej nazwie, ale można go łatwo obliczyć jako **max - min**.
- **Wariancja (Variance)**: Średnia z kwadratów odchyleń od średniej. Można ją obliczyć funkcją **var**.
- **Odchylenie standardowe (Std.Dev)**: Pierwiastek z wariancji. Można je obliczyć funkcją **std**.

In [ ]:
for col_name in ['Grade','StudyHours']:
    col = df_students[col_name]
    rng = col.max() - col.min()
    var = col.var()
    std = col.std()
    print('\n{}:\n - Range: {:.2f}\n - Variance: {:.2f}\n - Std.Dev: {:.2f}'.format(col_name, rng, var, std))

Spośród tych statystyk odchylenie standardowe jest zwykle najbardziej użyteczne. Daje miarę zmienności w tej samej skali co dane (punkty ocen dla kolumny Grade i godziny dla StudyHours). Im większe odchylenie standardowe, tym bardziej wartości odbiegają od średniej — mówiąc prosto, dane są bardziej rozproszone.

W przypadku rozkładu *normalnego* odchylenie standardowe wraz z właściwościami tego rozkładu daje dodatkowe wnioski. Uruchom poniższą komórkę, aby zobaczyć zależność między odchyleniami standardowymi a rozkładem normalnym.

In [ ]:
import scipy.stats as stats

# Get the Grade column
col = df_students['Grade']

# get the density
density = stats.gaussian_kde(col)

# Plot the density
col.plot.density()

# Get the mean and standard deviation
s = col.std()
m = col.mean()

# Annotate 1 stdev
x1 = [m-s, m+s]
y1 = density(x1)
plt.plot(x1,y1, color='magenta')
plt.annotate('1 std (68.26%)', (x1[1],y1[1]))

# Annotate 2 stdevs
x2 = [m-(s*2), m+(s*2)]
y2 = density(x2)
plt.plot(x2,y2, color='green')
plt.annotate('2 std (95.45%)', (x2[1],y2[1]))

# Annotate 3 stdevs
x3 = [m-(s*3), m+(s*3)]
y3 = density(x3)
plt.plot(x3,y3, color='orange')
plt.annotate('3 std (99.73%)', (x3[1],y3[1]))

# Show the location of the mean
plt.axvline(col.mean(), color='cyan', linestyle='dashed', linewidth=1)

plt.axis('off')

plt.show()

Poziome linie pokazują odsetek danych mieszczących się w jednym, dwóch i trzech odchyleniach standardowych od średniej (w obu kierunkach).

W dowolnym rozkładzie normalnym:

Skoro średnia ocen wynosi 49.18, a odchylenie standardowe 21.74, a rozkład ocen jest w przybliżeniu normalny, można obliczyć, że 68.26% studentów powinno uzyskać ocenę między 27.44 a 70.92.

Statystyki opisowe, których używamy do zrozumienia rozkładu zmiennych, są podstawą analizy statystycznej. Ponieważ są tak istotne przy eksploracji danych, obiekt DataFrame ma wbudowaną metodę `describe`, która zwraca podstawowe statystyki dla wszystkich kolumn numerycznych.
So, because we know that the mean grade is 49.18, the standard deviation is 21.74, and distribution of grades is approximately normal, we can calculate that 68.26% of students should achieve a grade between 27.44 and 70.92.

The descriptive statistics we've used to understand the distribution of the student data variables are the basis of statistical analysis. Because they're such an important part of exploring your data, there's a built-in `describe` method of the DataFrame object that returns the main descriptive statistics for all numeric columns.

In [ ]:
print(df_students.describe())

## Porównywanie danych

Skoro już wiemy coś o rozkładzie statystycznym danych w zbiorze, możemy zacząć szukać zależności między zmiennymi.

Najpierw usuńmy wiersze z wartościami odstającymi, aby otrzymać próbkę reprezentatywną dla typowej grupy studentów. Zidentyfikowaliśmy, że kolumna `StudyHours` zawiera kilka niezwykle niskich wartości, więc je wykluczymy.

In [ ]:
df_sample = df_students[df_students['StudyHours']>1]
print(df_sample)

### Porównanie zmiennych numerycznych i kategorycznych

Dane zawierają dwie zmienne *numeryczne* (`StudyHours` i `Grade`) oraz dwie *kategoryczne* (`Name` i `Pass`). Zacznijmy od porównania kolumny numerycznej `StudyHours` z kategoryczną `Pass`, aby sprawdzić, czy istnieje widoczna zależność między liczbą godzin nauki a zaliczeniem.

Aby to porównanie przeprowadzić, stwórzmy wykresy box pokazujące rozkład `StudyHours` dla każdej wartości `Pass` (true i false).

In [ ]:
df_sample.boxplot(column='StudyHours', by='Pass', figsize=(8,5))

Porównując rozkłady `StudyHours`, widać (co nie jest zaskakujące), że studenci, którzy zaliczyli, zwykle uczyli się dłużej niż ci, którzy nie zdali. Jeśli zatem chcesz przewidzieć, czy student zaliczy, liczba godzin nauki może być dobrym wskaźnikiem.

### Porównanie zmiennych numerycznych

Teraz porównajmy dwie zmienne numeryczne. Na początek stwórzmy wykres słupkowy pokazujący zarówno oceny, jak i godziny nauki.

In [ ]:
# Create a bar plot of name vs grade and study hours
df_sample.plot(x='Name', y=['Grade','StudyHours'], kind='bar', figsize=(8,5))

Wykres pokazuje słupki zarówno dla ocen, jak i godzin nauki dla każdego studenta, ale trudniej je porównać, ponieważ wartości są na różnych skalach. Oceny mierzy się w punktach (tu od 3 do 97), a czas nauki w godzinach (od 1 do 16).

Częstą techniką przy pracy z danymi numerycznymi na różnych skalach jest *normalizacja*, czyli przeskalowanie wartości tak, aby zachowały proporcje, ale były porównywalne. Użyjemy metody *MinMax*, która rozkłada wartości na przedział od 0 do 1. Można to zaimplementować samodzielnie, ale biblioteka **Scikit-Learn** zapewnia gotowy skalownik.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Get a scaler object
scaler = MinMaxScaler()

# Create a new dataframe for the scaled values
df_normalized = df_sample[['Name', 'Grade', 'StudyHours']].copy()

# Normalize the numeric columns
df_normalized[['Grade','StudyHours']] = scaler.fit_transform(df_normalized[['Grade','StudyHours']])

# Plot the normalized values
df_normalized.plot(x='Name', y=['Grade','StudyHours'], kind='bar', figsize=(8,5))

Po normalizacji danych łatwiej dostrzec zależność między oceną a czasem nauki. Nie jest to dokładne dopasowanie, ale wyraźnie widać, że studenci z wyższymi ocenami zwykle uczyli się więcej.

Wygląda więc na to, że istnieje korelacja między czasem nauki a oceną. Istnieje nawet statystyczna miara korelacji, której można użyć do ilościowego określenia tej zależności.

In [ ]:
print(df_normalized.Grade.corr(df_normalized.StudyHours))

Statystyka korelacji przyjmuje wartości między -1 a 1 i wskazuje siłę związku. Wartości powyżej 0 oznaczają korelację *dodatnią* (wysokie wartości jednej zmiennej współwystępują z wysokimi wartościami drugiej), natomiast wartości poniżej 0 oznaczają korelację *ujemną*.

W tym przypadku wartość korelacji jest bliska 1, co wskazuje na silną dodatnią zależność między czasem nauki a oceną.

> **Uwaga**: Dane pokazują korelację, ale korelacja nie oznacza przyczynowości. Innymi słowy, nie należy zakładać, że jedna zmienna powoduje zmianę drugiej tylko na podstawie korelacji. W naszym przykładzie statystyka pokazuje, że studenci z wysokimi ocenami zwykle spędzają więcej czasu na nauce, ale nie dowodzi to, że osiągnęli wysokie oceny *ponieważ* uczyli się więcej. Można równie dobrze błędnie twierdzić, że studenci uczyli się dużo *ponieważ* ich oceny miały być wysokie.

Innym sposobem wizualizacji korelacji między dwiema zmiennymi numerycznymi jest wykres *scatter* (punktowy).

In [ ]:
# Create a scatter plot
df_sample.plot.scatter(title='Study Time vs Grade', x='StudyHours', y='Grade')

Ponownie widoczny jest wzorzec: studenci, którzy uczyli się najwięcej godzin, zwykle osiągali najwyższe oceny.

Możemy to zobaczyć wyraźniej, dodając do wykresu linię regresji (linię najlepszego dopasowania), która pokazuje ogólny trend w danych. Zrobimy to za pomocą metody statystycznej zwanej *regresją metodą najmniejszych kwadratów*.

Pewnie pamiętasz ze szkoły równanie linii w postaci kierunkowej (slope-intercept):

*y = m x + b*

W równaniu tym *y* i *x* to zmienne współrzędne, *m* to nachylenie (slope), a *b* to wyraz wolny (punkt przecięcia z osią Y).

W naszym przypadku mamy x = `StudyHours` i y = `Grade`. Musimy obliczyć współczynnik kierunkowy i wyraz wolny dla linii, która jest jak najbliżej punktów. Dla każdego x obliczamy wartość *f(x)* na tej linii, a różnica między rzeczywistą wartością y (Grade) a *f(x)* to błąd dopasowania. Celem jest znalezienie linii minimalizującej sumę kwadratów tych błędów.

Szczegółowo: dla każdego punktu liczony jest błąd, następnie błąd ten jest podnoszony do kwadratu, a suma tych kwadratów jest minimalizowana — stąd nazwa *methoda najmniejszych kwadratów*.

Na szczęście nie musisz implementować tego samodzielnie — pakiet **SciPy** posiada klasę **stats** z metodą **linregress**, która zwraca współczynniki potrzebne do wyznaczenia nachylenia (*m*) i wyrazu wolnego (*b*).

In [ ]:
from scipy import stats

#
df_regression = df_sample[['Grade', 'StudyHours']].copy()

# Get the regression slope and intercept
m, b, r, p, se = stats.linregress(df_regression['StudyHours'], df_regression['Grade'])
print('slope: {:.4f}\ny-intercept: {:.4f}'.format(m,b))
print('so...\n f(x) = {:.4f}x + {:.4f}'.format(m,b))

# Use the function (mx + b) to calculate f(x) for each x (StudyHours) value
df_regression['fx'] = (m * df_regression['StudyHours']) + b

# Calculate the error between f(x) and the actual y (Grade) value
df_regression['error'] = df_regression['fx'] - df_regression['Grade']

# Create a scatter plot of Grade vs StudyHours
df_regression.plot.scatter(x='StudyHours', y='Grade')

# Plot the regression line
plt.plot(df_regression['StudyHours'],df_regression['fx'], color='cyan')

# Display the plot
plt.show()

Zauważ, że tym razem wykres zawiera dwa elementy: wykres punktowy (StudyHours vs Grade) oraz linię najlepszego dopasowania opartą na współczynnikach regresji.

Współczynniki nachylenia i wyrazu wolnego obliczone dla linii regresji są wyświetlone powyżej wykresu.

Linia opiera się na wartościach *f(x)* obliczonych dla każdej wartości `StudyHours`. Uruchom następną komórkę, aby zobaczyć tabelę zawierającą:

- `StudyHours` dla każdego studenta
- `Grade` uzyskaną przez studenta
- obliczoną wartość *f(x)* na podstawie współczynników regresji
- *błąd* między obliczoną wartością *f(x)* a rzeczywistą wartością `Grade`

Niektóre błędy, zwłaszcza na skrajach, są dość duże (ponad 17 punktów ocen). Ogólnie jednak linia jest całkiem bliska rzeczywistym ocenom.

In [ ]:
# Show the original x,y values, the f(x) value, and the error
print(df_regression[['StudyHours', 'Grade', 'fx', 'error']])

### Wykorzystanie współczynników regresji do prognozy

Mając współczynniki regresji opisujące zależność między czasem nauki a oceną, możesz użyć ich w funkcji do oszacowania oczekiwanej oceny dla podanej liczby godzin nauki.

In [ ]:
# Define a function based on our regression coefficients
def f(x):
    m = 6.3134
    b = -17.9164
    return m*x + b

study_time = 14

# Get f(x) for study time
prediction = f(study_time)

# Grade can't be less than 0 or more than 100
expected_grade = max(0,min(100,prediction))

#Print the estimated grade
print ('Studying for {} hours per week may result in a grade of {:.0f}'.format(study_time, expected_grade))

Zastosowanie metod statystycznych do próbek pozwoliło określić zależność między czasem nauki a oceną oraz zamknąć ją w ogólnej funkcji, którą można użyć do przewidywania oceny dla danej liczby godzin nauki.

Ta technika jest w istocie podstawą uczenia maszynowego. Na podstawie próbek zawierających jedną lub więcej *cech* (tutaj liczba godzin nauki) oraz znanej *etykiety* (tutaj otrzymana ocena) możemy wyprowadzić funkcję przewidującą etykiety dla nowych danych wejściowych.

## Podsumowanie

W tym notatniku omówiliśmy:

1. Czym jest wartość odstająca i jak ją usuwać
2. Jak dane mogą być skośne
3. Jak badać rozrzut danych
4. Podstawowe sposoby porównywania zmiennych, takie jak oceny i czas nauki
